# Installing Dependencies

In [ ]:
!pip install -r requirements.txt

# Importing Required Libraries

In [1]:
import os
import sys

import numpy as np
import pandas as pd

import osmnx as ox
import networkx as nx

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain.tools import BaseTool

from typing import Type
from pydantic import BaseModel, Field

# Auxiliar Functions Definition

In [2]:
def haversine(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
        Compute the haversine distance between two points in latitude and longitude.
        The haversine formula determines the great-circle distance between two points on a sphere given their longitudes and latitudes.

        Parameters
        ----------
        lat1, lon1, lat2, lon2 : float
            Latitude and longitude of the two points in decimal degrees.

        Returns
        -------
        distance : float
            Distance between the two points in kilometers.
    """
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Earth radius in kilometers

    return c * r

def find_nearest_risk_point(lat: float, lon: float, data: pd.DataFrame) -> int:
    """
        Find the nearest risk level point to a given latitude and longitude.

        Parameters
        ----------
        lat, lon : float
            Latitude and longitude of the point.
        data : pd.DataFrame
            DataFrame containing the risk level points with columns 'lat', 'lon' and 'RL'.

        Returns
        -------
        RL : int
            Risk level of the nearest point.
    """
    distances = data.apply(lambda row: haversine(lat, lon, row['lat'], row['lon']), axis=1)

    nearest_index = distances.idxmin()
    
    return data.loc[nearest_index, 'RL']

# LangChain Tools Definition

In [3]:
class RiskPointInput(BaseModel):
    lat: float = Field(..., title="Latitude", description="Latitude of the point")
    lon: float = Field(..., title="Longitude", description="Longitude of the point")

class RiskPointTool(BaseTool):
    name = "RiskPointTool"
    description = "A tool that computes the risk of a point in the map"
    args_schema: Type[BaseModel] = RiskPointInput

    def _run(
        self,
        lat: float,
        lon: float
    ) -> str:
        data = pd.read_csv('risk_map.csv', sep=";")
        risk_lane = find_nearest_risk_point(lat, lon, data)

        return risk_lane

In [4]:
class RiskTripInput(BaseModel):
    start_lat: float = Field(..., title="Latitude of the starting point", description="Latitude of the starting point")
    start_lon: float = Field(..., title="Longitude of the starting point", description="Longitude of the starting point")
    end_lat: float = Field(..., title="Latitude of the ending point", description="Latitude of the ending point")
    end_lon: float = Field(..., title="Longitude of the ending point", description="Longitude of the ending point")

class RiskTripTool(BaseTool):
    name = "RiskTripTool"
    description = "A tool that computes the risk of a trip in the map"
    args_schema: Type[BaseModel] = RiskTripInput

    def _run(
        self,
        start_lat: float,
        start_lon: float,
        end_lat: float,
        end_lon: float
    ) -> str:
        data = pd.read_csv('risk_map.csv', sep=";")
        place_name = "Porto, Portugal"
        graph = ox.graph_from_place(place_name, network_type='all')

        start = (start_lat, start_lon)
        end = (end_lat, end_lon)

        start_node = ox.distance.nearest_nodes(graph, start[1], start[0])
        end_node = ox.distance.nearest_nodes(graph, end[1], end[0])

        route = nx.shortest_path(graph, start_node, end_node, weight='length')

        risk = {
            1: 0,
            2: 0,
            3: 0
        }

        for node in route:
            node_data = graph.nodes[node]
            risk_lane = find_nearest_risk_point(node_data['y'], node_data['x'], data)
            risk[risk_lane] += 1

        sum_risk = risk[1] + risk[2] + risk[3]

        return f"Low risk percentage: {risk[1]/sum_risk*100:.2f}%, Medium risk percentage: {risk[2]/sum_risk*100:.2f}%, High risk percentage: {risk[3]/sum_risk*100:.2f}%"

# Set OpenAI API Key

In [6]:
openai_api_key = input("Please enter your OpenAI API key: ")

# Creating Agent

## Definig the Model

In [12]:
chat_open_ai = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=openai_api_key)

## Instantiating the Tools

In [ ]:
risk_point_tool = RiskPointTool()
risk_trip_tool = RiskTripTool()

tools = [
    risk_point_tool,
    risk_trip_tool
]

## Creating the Prompt

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are an agent responsible for assisting users in computing cycling routes based on risk levels. Your task is to analyze the computed risk levels and provide comprehensive and well-tailored responses to user inquiries.

            Risk Levels:
            - (1) Low Risk: Routes that are safe for most cyclists, including beginners. These routes are well within the city's emergency mitigation capacity, with close proximity to emergency services.
            - (2) Medium Risk: Routes that may have moderate distance from emergency services. These routes are suitable for experienced cyclists who are comfortable with occasional risks and may require emergency services less readily available.
            - (3) High Risk: Routes that are far from emergency services and other emergency-related amenities. These routes involve significant hazards and are recommended only for very experienced cyclists or those seeking a challenge.

            Main Instructions:
            1. Input Interpretation:
            - Receive the computed risk level (1, 2, or 3) from the risk analysis system.
            - Understand the context and specifics of each risk level as detailed above, focusing on the city's emergency mitigation capacity.

            2. Response Generation:
            - Provide clear and detailed descriptions of the route based on the given risk level.
            - Include relevant safety tips and precautions specific to the risk level.
            - Suggest the user to look for safer alternative routes if the computed risk level is considered high (the route passes througout many high-risk areas).

            3. Example Responses:
            - Low Risk (1): "The route from point A to point B is considered low risk, making it ideal for beginners. The path is within close proximity to emergency services. Ensure you stay on designated cycling lanes for maximum safety."
            - Medium Risk (2): "The route from point A to point B has a medium risk level, suitable for experienced cyclists. While not immediately near emergency services, they are accessible with some delay. Be cautious at intersections and consider wearing high-visibility gear."
            - High Risk (3): "The route from point A to point B is high risk, recommended only for very experienced cyclists. The path includes heavy traffic and is far from emergency services. For a safer alternative, consider another route and provide the information that I will assist you in evaluating it to assure you stay safe."

            4. General Tips:
            - Always remind users to wear appropriate safety gear, such as helmets and reflective clothing - reinforce additional gear if the route has high risk.
            - Encourage users to check weather conditions before starting their journey.
            - Advise users to carry a basic repair kit and stay hydrated.
            - Suggest carrying a mobile phone and sharing route details with someone in case of an emergency.

            By following these guidelines, you will ensure that users receive precise, helpful, and contextually appropriate information for their cycling routes. Your goal is to enhance their cycling experience while prioritizing their safety based on the city's emergency mitigation capacity.
            """
        ),
        (
            "user",
            "{input}"
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

## Binding the Tools to the Model

In [ ]:
llm_with_tools = chat_open_ai.bind_tools(tools)

## Creating the Chain

In [ ]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
    }
    | prompt_template
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

## Creating the Agent Executor

In [13]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

# Running

## Example 1

In [8]:
user_input = "What is the risk of a cycling trip starting from point with latitude 41.1246 and longitude -8.5717, ending at latitude 41.1570 and longitude -8.6393?"

In [9]:
result = list(
    agent_executor.stream(
        {"input": user_input}
    )
)

print(result[-1]["messages"][0].content)

The computed risk levels for the cycling trip from the starting point at latitude 41.1246 and longitude -8.5717 to the ending point at latitude 41.1570 and longitude -8.6393 are as follows:
- Low Risk: 53.79%
- Medium Risk: 30.30%
- High Risk: 15.91%

Based on these risk percentages, the trip is categorized as having a low risk level. It is considered safe for most cyclists, including beginners. The path is within close proximity to emergency services, ensuring a safer journey. Remember to stay on designated cycling lanes for maximum safety and enjoy your ride!


## Example 2

In [10]:
user_input = "I am planning a cycling trip that will cross the point at latitude 41.1246 and longitude -8.5717. What is the risk associate to this zone?"

In [11]:
result = list(
    agent_executor.stream(
        {"input": user_input}
    )
)

print(result[-1]["messages"][0].content)

The point at latitude 41.1246 and longitude -8.5717 is associated with a high risk level. This indicates that the area is far from emergency services and involves significant hazards. It is recommended only for very experienced cyclists or those seeking a challenge.

Considering the high risk level, I suggest looking for a safer alternative route to ensure your safety during the cycling trip. If you provide me with the starting and ending points of your trip, I can assist you in evaluating the risk level of the entire route and help you find a safer path.
